In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())

In [3]:
spark

In [4]:
data_folder = '/labs/slaba02'
file_name = 'DO_record_per_line.json'

path_to_file = os.path.join(data_folder, file_name)

In [5]:
! hdfs dfs -ls /labs/slaba02

Found 1 items
-rw-r--r--   3 hdfs hdfs   69519728 2022-01-06 18:46 /labs/slaba02/DO_record_per_line.json


In [6]:
data = spark.sparkContext.textFile(path_to_file)
data.first()

'{"lang": "en", "name": "Accounting Cycle: The Foundation of Business Measurement and Reporting", "cat": "3/business_management|6/economics_finance", "provider": "Canvas Network", "id": 4, "desc": "This course introduces the basic financial statements used by most businesses, as well as the essential tools used to prepare them. This course will serve as a resource to help business students succeed in their upcoming university-level accounting classes, and as a refresher for upper division accounting students who are struggling to recall elementary concepts essential to more advanced accounting topics. Business owners will also benefit from this class by gaining essential skills necessary to organize and manage information pertinent to operating their business. At the conclusion of the class, students will understand the balance sheet, income statement, and cash flow statement. They will be able to differentiate between cash basis and accrual basis techniques, and know when each is appr

In [7]:
data = spark.read.format('org.apache.spark.sql.json').load(path_to_file)
data.show()

+--------------------+--------------------+---+----+--------------------+--------------+
|                 cat|                desc| id|lang|                name|      provider|
+--------------------+--------------------+---+----+--------------------+--------------+
|3/business_manage...|This course intro...|  4|  en|Accounting Cycle:...|Canvas Network|
|              11/law|This online cours...|  5|  en|American Counter ...|Canvas Network|
|5/computer_scienc...|This course is ta...|  6|  fr|Arithmétique: en ...|Canvas Network|
|  14/social_sciences|We live in a digi...|  7|  en|Becoming a Dynami...|Canvas Network|
|2/biology_life_sc...|This self-paced c...|  8|  en|           Bioethics|Canvas Network|
|9/humanities|15/m...|This game-based c...|  9|  en|College Foundatio...|Canvas Network|
|  14/social_sciences|What’s in your di...| 10|  en|Digital Literacies I|Canvas Network|
|  14/social_sciences|The goal of the D...| 11|  en|Digital Literacie...|Canvas Network|
|  14/social_sciences

In [71]:
data = data.filter((col("desc") != ' ') & (col("desc") != '\n\t  \t     \n\t  \t  '))

In [72]:
import re

STOP_WORDS = ['', 'о', 'на', 'для', 'со', 'до', 'в',
              'and', 'or']

def clean_list_of_tokens(tokens):
    new_tokens = []
    for token in tokens:
        token = re.sub('[\W\d_]+', '', token)
        if token not in STOP_WORDS:
            new_tokens.append(token)
    return new_tokens

In [74]:
from pyspark.sql.functions import UserDefinedFunction, col, array
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, RegexTokenizer, StopWordsRemover
from pyspark.sql.functions import explode
from pyspark.ml.linalg import DenseVector
from pyspark.ml.feature import BucketedRandomProjectionLSH

import numpy as np


def pipeline(course_id: str):

    _lng = data.filter(col("id") == course_id).select("lang").collect()[0].__getitem__('lang')
    _data_lng = data.filter(data.lang == _lng)
    tokenizer = Tokenizer(inputCol="desc", outputCol="words")
    tokenized = tokenizer.transform(_data_lng)

    remover = StopWordsRemover(inputCol="words", outputCol="wordsFiltered")
    tokenized_wo_stopwords = remover.transform(tokenized)

    convertUDF = UserDefinedFunction(lambda x: clean_list_of_tokens(x), ArrayType(StringType()))
    tokenized_wo_stopwords = tokenized_wo_stopwords.withColumn("wordsFiltered", convertUDF(col("wordsFiltered")))

    hashingTF = HashingTF(inputCol="wordsFiltered", outputCol="rawFeatures", numFeatures=1000)  # wordsFiltered
    featurizedData = hashingTF.transform(tokenized_wo_stopwords)

    idf = IDF(inputCol="rawFeatures", outputCol="features")
    idfModel = idf.fit(featurizedData)
    rescaledData = idfModel.transform(featurizedData)

    tf_idf_vectors = rescaledData.select("features", "id").rdd.map(lambda x: (x.id, DenseVector(x.features.toArray())))
    df = spark.createDataFrame(tf_idf_vectors,)

    one_row = df.where(df._1 == course_id).first().asDict()['_2']

    def cos_sim(a, b=one_row):
        return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))

    # apply the UDF to the column
    cosSumUDF = UserDefinedFunction(lambda x: cos_sim(x), FloatType())
    df = df.withColumn("coSim", cosSumUDF(col("_2")))
    return df.filter(col("_1") != course_id).sort("coSim", ascending = False).take(TOP_N)

In [75]:
from collections import defaultdict
import json

TEST_IDS = ["23126", "21617", "16627", "11556", "16704", "13702"]
TOP_N = 10

my_recommendations = defaultdict(list)
for course_id in TEST_IDS:
    result = pipeline(course_id)
    for course in result:
        if str(course._1) != course_id:
            my_recommendations[course_id].append(course._1)
            
result = json.dumps(my_recommendations)

In [95]:
result = {"23126": [13727, 10764, 13665, 14760, 24419, 5114, 9949, 13782, 26804, 23478], "21617": [21609, 21608, 21616, 21492, 21624, 21700, 21623, 21630, 21628, 21508], "16627": [11431, 17964, 5687, 17961, 16694, 12660, 12247, 5558, 11575, 9598], "11556": [10384, 16488, 11605, 468, 22710, 19330, 13461, 11523, 19279, 23357], "16704": [1250, 1164, 21482, 1411, 1247, 978, 913, 20095, 927, 20096], "13702": [864, 1111, 1052, 895, 1054, 929, 8300, 1033, 894, 1383]}
# result = {"23126": [2909, 2633, 17499, 3382, 2495, 673, 13130, 353, 3576, 13782], "21617": [21609, 16414, 3929, 8132, 22302, 14545, 15968, 2792, 26698, 27346], "16627": [27556, 27518, 27313, 26196, 27174, 27091, 26521, 26949, 21695, 26738], "11556": [9760, 3901, 10491, 26987, 27391, 9761, 25941, 28256, 26199, 26426], "16704": [8312, 999, 1000, 8311, 951, 28005, 17195, 19614, 1246, 814], "13702": [864, 1161, 8317, 1301, 26501, 1119, 8176, 1225, 1236, 27087]}
# result = {"23126": [16709, 2909, 17499, 3382, 1587, 24419, 2495, 1607, 7993, 14760], "21617": [21609, 21687, 1908, 21587, 15427, 26872, 2110, 1551, 2822, 2177], "16627": [13551, 21704, 13552, 21705, 21695, 13529, 27556, 27518, 26949, 9044], "11556": [9760, 26738, 26274, 27091, 25941, 26697, 10491, 9761, 27391, 27062], "16704": [25916, 6928, 1291, 1418, 20355, 1069, 20301, 1406, 8800, 22210], "13702": [17017, 17018, 17071, 17019, 17020, 17010, 17021, 17013, 17022, 17015]}
# result = {"23126": [23564, 24885, 20045, 18998, 12307, 24488, 24091, 4322, 9549, 18089], "21617": [21609, 23567, 12683, 4962, 5334, 18970, 21601, 21660, 4388, 21548], "16627": [23557, 20610, 7668, 23515, 25797, 5463, 7250, 11500, 20126, 22735], "11556": [22629, 7384, 23597, 12200, 11510, 21838, 2468, 20817, 10912, 9983], "16704": [901, 25920, 12201, 12725, 1269, 8790, 882, 17125, 17119, 6928], "13702": [775, 786, 776, 774, 778, 779, 574, 785, 780, 767]}

In [96]:
import json
with open("lab02.json", "w") as json_data:
    json.dump(result, json_data)

In [94]:
test_course_id = "13702"

print(data.filter(col("id") == test_course_id).select("desc").collect())
for course in result[test_course_id]:
    print(data.filter(col("id") == str(course)).select("id", "desc").collect())

[Row(desc='Математическая экономика – это набор моделей в той или иной степени правильно описывающих процессы в экономике.')]
[Row(id=895, desc='В компьютерном моделировании значительную роль занимает графическое представление моделируемых объектов, явлений и процессов. Графические модели позволяют не только более наглядно представить моделируемые сущности, но  и упростить интерпретацию полученных результатов.  Учебный курс предназначен для изучения базовых возможностей графической библиотеки OpenGL и получения практических навыков создания графических моделей различных типов: от графиков до реалистичных моделей объектов.')]
[Row(id=28074, desc='11 лекций филолога Гасана Гусейнова о точках пересечения современной культуры и древнего мифа\n \n\n \n Почти во всех европейских языках основной лексикон, используемый для представления знания, позаимствован у греков и римлян. Но есть одно слово, которое люди уже не одну тысячу лет оставляют в своих языках в его оригинальной греческой форме, —

[Row(id=1266, desc='Предлагаемый курс рассматривает различные технические и творческие аспекты цифровой фотографии, позволяя слушателям научиться получать технически совершенные снимки и освоить базовые навыки в области творческой светописи.')]
[Row(id=1111, desc='Теория экономических механизмов (mechanism design theory) — быстроразвивающаяся и относительно молодая область экономики, направленная на создание механизмов взаимодействия между эгоистичными агентами.')]
[Row(id=1077, desc='Предметом курса являются архитектуры, технологии и инструментарий создания корпоративных систем, т.е. структурная организация таких систем, схемы и средства их промышленного производства.')]
[Row(id=1042, desc='Предметом курса являются примеры построения реализаций корпоративных систем, с учетом характерных особенностей различных предметных областей. Акцент сделан на программную платформу Microsoft Dynamics и инструментарий Oracle.')]
[Row(id=8123, desc='Простое и комфортное введение в язык программирован

In [97]:
spark.stop()